In [2]:
import pandas as pd
from  icd import ICD

Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/virtual_assistant/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at emilyalsentzer/Bio_Discharge_Summary_BERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Be

In [3]:
model = ICD()

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


ICD Prediction : Initializing
ICD Prediction : Initializing : Finished in   1.498 Seconds



In [4]:
input_text = 'hi my name is dionna'
prediction = model.predict(text=input_text)

ICD Prediction : Predicting ICD Codes
Data Embeddings time : 0.203291654586792


/home/ec2-user/mlops-nlp-healthcare-domain/pretrained_models/icd_predict/ClinicalBERTEmbeddings.py:102: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tokens = torch.tensor(y).to(torch.int64)


Data Embeddings padding : 0.3720881938934326
Chapter Pred: 1.2509069442749023
Adverse Pred: 2.112053871154785
selected models: ['390_459', '280_289', '240_279', '520_579', '580_629', '001_139', '710_739', '800_999', '780_799', '290_319', '680_709', 'E_V', '460_519', '140_239', '320_389', '630_679__740_759__760_779']
All Other Models Pred: 15.118667840957642
ICD Prediction : Predicting ICD Codes : Finished in  15.119 Seconds


In [13]:
data = ClinicalBERTEmbeddings(input_text)

embeddings_padded = []
for j in data:
    pad_len = 5120 - len(j)
    if pad_len:
        embeddings_padded.append(np.append(j, np.zeros(pad_len*768, dtype=np.float16).reshape(pad_len, 768), axis=0))
    else:
        embeddings_padded.append(j)
        
embeddings_padded = np.float16(embeddings_padded)
embeddings_padded = np.array(embeddings_padded, dtype=np.float16)
data.shape

(20, 256, 768)

In [12]:
embeddings_padded.shape

(20, 5120, 768)

In [16]:
# Chapter Model
PATH = './'
json_file = open(PATH + 'models/models/chapter/chapter_num.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
chapter_model.load_weights(PATH + "models/models/chapter/chapter_model.h5")
#self.chapter_labels = pd.read_csv(PATH + 'models/models/chapter/chapter_label.csv').columns[1:]

In [18]:
chapter_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5120, 768)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5113, 16)          98320     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1278, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20448)             0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 20448)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               2617472   
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0   

In [21]:
model_dict = {}
for file in glob.glob(PATH + "models/models/disease/*.json"):  
    baseName = os.path.basename(file)[6:-9]   
    json_file = open(file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_dict[baseName] = model_from_json(loaded_model_json)
    model_dict[baseName].load_weights(file[:-5]+".h5")
    print(model_dict[baseName].summary())
    

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5120, 768)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5113, 16)          98320     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1278, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20448)             0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 20448)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)               2617472   
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0  

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5120, 768)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5113, 16)          98320     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1278, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20448)             0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 20448)             0         
_________________________________________________________________
dense_67 (Dense)             (None, 256)               5234944   
_________________________________________________________________
dropout_37 (Dropout)         (None, 256)               0  

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5120, 768)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5113, 16)          98320     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1278, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20448)             0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 20448)             0         
_________________________________________________________________
dense_88 (Dense)             (None, 128)               2617472   
_________________________________________________________________
dropout_52 (Dropout)         (None, 128)               0  